# Final Capstone Project Report for the IBM course in Coursera | Battle of the Neighborhoods in Middelburg


&copy; Teun Huibregtse

## Preface

*In Dutch:

Dit bestand (notebook) en bijbehorende bijdragen is het resultaat van mijn laatste IBM Course voor Datascience, in dit notebook mag ik mijn eigen probleemdefinitie schrijven en hier tevens een oplossing voor bieden, het is echter noodzakelijk om bij het verzamelen van de data gebruik te maken van de Foursquare API, de API kan gebruikt worden voor het ophalen van interesses en beoordelingen van bijvoorbeeld restaurants of de lokale sportschool binnen een eigen gedefinieerde radius of het ophalen van gebruikersdata van Foursquare profielen, het uiteindelijke doel van de course is de mogelijkheid om problemen te beantwoorden van stakeholders met vraagstellingen als:

- Als ik zou verhuizen naar een plaats dichter bij mijn werk, naar welke wijk zou ik dan het beste naar toe kunnen verhuizen op basis van mijn intresses en behoeften?




## 1. Introduction/Business Problem (A description of the problem and a discussion of the background. )

*pre note: My English grammar and language usage isn't the best, so this isn't a professional document as I would prefer to do it in Dutch, however to pass the assignment I will do it in English.

*For a more detailed description see the full report available in the github repository, as this document is for showing code only

As i'm not a creative person myself either, I would like to tackle the predefined problem defined in the course objective.

<strong>In this project I will answer the following question:</strong>

- In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it?

The city of my choice will be Middelburg, as this is my favorite city in Zeeland, The Netherlands.



## 2. Data (A description of the data and how it will be used to solve the problem. )

The following types of data will be used: 
- List of neighborhoods in Middelburg (https://nl.wikipedia.org/wiki/Wijken_en_buurten_in_Middelburg)
- maps.ie for finding coordinates (https://www.maps.ie/coordinates.html)
- Foursquare API for scraping and gathering relevant information


## Output

### Installing needed packages and libraries

In [25]:
# installing packages from system
import sys
!{sys.executable} -m pip install folium

# installing libraries
import requests
import folium
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

# IBM watson studio specific libraries
import types
from botocore.client import Config
import ibm_boto3

print()
print('Packages and Libraries installed!')


Packages and Libraries installed!


### Generating and preparing the data

The neighborhoods `csv` file will be created manualy as there is no direct downloadable data available, the data is generated with:
- maps.ie for the coordinates [https://www.maps.ie/coordinates.html](https://www.maps.ie/coordinates.html)
- wikipedia/CBS for the list of neighborhoods [https://nl.wikipedia.org/wiki/Wijken_en_buurten_in_Middelburg](https://nl.wikipedia.org/wiki/Wijken_en_buurten_in_Middelburg)

### lets explore and show the data of the neighborhoods of Middelburg lets explore and show the data of the neighborhoods of Middelburg

In [26]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_1e19b74757f24a8b83de9e68289ba7ea = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='HIDDEN',
    ibm_auth_endpoint="HIDDEN",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_1e19b74757f24a8b83de9e68289ba7ea.get_object(Bucket='capstoneprojectnotebook-donotdelete-pr-t8lup5iwaxcjuo',Key='neighborhoods_middelburg.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_mburg_neigh = pd.read_csv(body)
print('shape: ', df_data_mburg_neigh.shape)
df_data_mburg_neigh


shape:  (11, 3)


,Neighborhood,Latitude,Longitude
0,Binnenstad,51.499732,3.613752
1,Griffioen,51.506202,3.595598
2,Klarenbeek,51.508051,3.610783
3,Nieuw Middelburg,51.506233,3.625254
4,Veersepoort,51.512197,3.622476
5,Stromenwijk 't Zand,51.491164,3.599305
6,Middelburg-Zuid,51.484007,3.614082
7,Dauwendaele,51.492077,3.625996
8,Mortiere,51.483405,3.635964
9,Nieuw - en Sint Joosland,51.481970,3.656884


### Visualizing the neighborhoods using lat/long on a map

In [27]:
# set latitude and longtitude of the city of Middelburg in The Netherlands
lat_long_mburg_nl = ('51.4997302', '3.6136962')

map_of_mburg_nl = folium.Map(location=lat_long_mburg_nl, zoom_start=13)

# set markers on the folium map
for lat, lng, label in zip(df_data_mburg_neigh['Latitude'],
                           df_data_mburg_neigh['Longitude'],
                           df_data_mburg_neigh['Neighborhood']):
    
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#03ecfc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_of_mburg_nl)  
    
map_of_mburg_nl

### Setup and explore the Foursquare API

In [28]:
# Setup credentials
CLIENT_ID = 'HIDDEN'
CLIENT_SECRET = 'HIDDEN'
VERSION = '20180605' # Foursquare API version

print('Foursquare API credentails:')
print('VERSION: ', VERSION)
print('CLIENT_ID: ', CLIENT_ID)
print('CLIENT_SECRET: ', CLIENT_SECRET)

Foursquare API credentails:
VERSION:  20180605
CLIENT_ID:  RSNAO1FQOALHEEXTDEOLEVWTPPKV10H2XQEEGMNFH0AIY1Q5
CLIENT_SECRET:  GCMQTGRALCVDTRZWCLRVFY5GS3LD30GBCOSIQIIRRP04U4RB


In [29]:
# Setup API 
LIMIT = 100
radius = 800

neighborhood_latitude = df_data_mburg_neigh['Latitude'][0]
neighborhood_longitude = df_data_mburg_neigh['Longitude'][0]

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

print('URL of Foursquare API: ', url )

URL of Foursquare API:  https://api.foursquare.com/v2/venues/explore?&client_id=RSNAO1FQOALHEEXTDEOLEVWTPPKV10H2XQEEGMNFH0AIY1Q5&client_secret=GCMQTGRALCVDTRZWCLRVFY5GS3LD30GBCOSIQIIRRP04U4RB&v=20180605&ll=51.49973189999999,3.6137522000000004&radius=800&limit=100


In [30]:
# send request and test results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dd55fed0be7b4001b329f66'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Middelburg',
  'headerFullLocation': 'Middelburg',
  'headerLocationGranularity': 'city',
  'totalResults': 67,
  'suggestedBounds': {'ne': {'lat': 51.5069319072, 'lng': 3.625296541714101},
   'sw': {'lat': 51.49253189279999, 'lng': 3.6022078582858996}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bdaf85863c5c9b6dba92568',
       'name': "B'tje Anders",
       'location': {'address': 'Reigerstraat 3',
        'lat': 51.499650648136054,
        'lng': 3.6158709231274946,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.499650648136054,
          'l

## Explore all Neighborhoods

In [31]:
# Define function to automate exploration through the Foursquare API
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['categories'][0]['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Category',
                  'Venue Latitude', 
                  'Venue Longitude']
    
    return(nearby_venues)

In [32]:
# Set and run function for each neighborhood
mburg_venues = getNearbyVenues(names=df_data_mburg_neigh['Neighborhood'],
                                   latitudes=df_data_mburg_neigh['Latitude'],
                                   longitudes=df_data_mburg_neigh['Longitude'],
                               radius=800
                                  )

Binnenstad
Griffioen
Klarenbeek
Nieuw Middelburg
Veersepoort
Stromenwijk 't Zand
Middelburg-Zuid
Dauwendaele
Mortiere
Nieuw - en Sint Joosland
Arnemuiden


In [33]:
# Show and test venue table
print(mburg_venues.shape)
mburg_venues.head()

(183, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,Binnenstad,51.499732,3.613752,B'tje Anders,Fast Food Restaurant,51.499651,3.615871
1,Binnenstad,51.499732,3.613752,De Drukkerij,Bookstore,51.497921,3.611670
2,Binnenstad,51.499732,3.613752,Markt,Plaza,51.498415,3.611665
3,Binnenstad,51.499732,3.613752,St John,Café,51.497060,3.614056
4,Binnenstad,51.499732,3.613752,Biercafé De Vliegende Hollander,Bar,51.500203,3.617550


In [34]:
# show count of venues inside neighborhoods
mburg_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
Neighborhood,,,,,,
Arnemuiden,9,9,9,9,9,9
Binnenstad,67,67,67,67,67,67
Dauwendaele,18,18,18,18,18,18
Griffioen,5,5,5,5,5,5
Klarenbeek,8,8,8,8,8,8
Middelburg-Zuid,12,12,12,12,12,12
Mortiere,18,18,18,18,18,18
Nieuw - en Sint Joosland,4,4,4,4,4,4
Nieuw Middelburg,19,19,19,19,19,19


In [35]:
# print unique venue categories inside venues table
print('There are {} unique venue categories in {} neighborhoods'.format(
    len(mburg_venues['Venue Category'].unique()),  
    len(mburg_venues['Neighborhood'].unique())))

There are 76 unique venue categories in 11 neighborhoods


## Data pre-processing

In [36]:
# one hot encoding
mburg_onehot = pd.get_dummies(mburg_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mburg_onehot['Neighborhood'] = mburg_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mburg_onehot.columns[-1]] + list(mburg_onehot.columns[:-1])
mburg_onehot = mburg_onehot[fixed_columns]

# show and test newly generated mburg_onehot table
print('Shape of mburg inside one hot encoding variable: ', mburg_onehot.shape)
mburg_onehot.head()

Shape of mburg inside one hot encoding variable:  (183, 77)


,Neighborhood,Arcade,Athletics & Sports,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Boat or Ferry,...,Sports Club,Supermarket,Tea Room,Theater,Theme Park,Toy / Game Store,Train Station,Video Store,Wine Shop,Yoga Studio
0,Binnenstad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Binnenstad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Binnenstad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Binnenstad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Binnenstad,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# show and test newly generated mburg_grouped table, check the size of the grouped dataframe
mburg_grouped = mburg_onehot.groupby('Neighborhood').mean().reset_index()
print('shape of mburg_grouped: ', mburg_grouped.shape)
mburg_grouped

shape of mburg_grouped:  (11, 77)


,Neighborhood,Arcade,Athletics & Sports,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Boat or Ferry,...,Sports Club,Supermarket,Tea Room,Theater,Theme Park,Toy / Game Store,Train Station,Video Store,Wine Shop,Yoga Studio
0,Arnemuiden,0.0,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,...,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
1,Binnenstad,0.0,0.000000,0.014925,0.029851,0.074627,0.000000,0.014925,0.014925,0.014925,...,0.0,0.014925,0.014925,0.014925,0.000000,0.000000,0.014925,0.000000,0.000000,0.014925
2,Dauwendaele,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000
3,Griffioen,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Klarenbeek,0.0,0.125000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.125000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000
5,Middelburg-Zuid,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Mortiere,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.055556,0.111111,0.000000,0.000000,0.000000,0.000000
7,Nieuw - en Sint Joosland,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Nieuw Middelburg,0.0,0.052632,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.052632,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000
9,Stromenwijk 't Zand,0.0,0.100000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,...,0.1,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [38]:
# print the top 5 most common venues inside each neighborhood
num_top_venues = 5

for hood in mburg_grouped['Neighborhood']:
    print(hood,":")
    temp = mburg_grouped[mburg_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','frequency']
    temp = temp.iloc[1:]
    temp['frequency'] = temp['frequency'].astype(float)
    temp = temp.round({'frequency': 2})
    print(temp.sort_values('frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Arnemuiden :
                        venue  frequency
0                         Gym       0.11
1                 Gas Station       0.11
2  Construction & Landscaping       0.11
3                Soccer Field       0.11
4                  Food Court       0.11


Binnenstad :
                 venue  frequency
0                 Café       0.10
1           Restaurant       0.07
2                  Bar       0.07
3                Hotel       0.06
4  Sporting Goods Shop       0.03


Dauwendaele :
                    venue  frequency
0                   Hotel       0.22
1                Bus Stop       0.11
2            Intersection       0.06
3  Furniture / Home Store       0.06
4             Fish Market       0.06


Griffioen :
                  venue  frequency
0  Fast Food Restaurant        0.4
1                Arcade        0.2
2    Chinese Restaurant        0.2
3           Supermarket        0.2
4                Museum        0.0


Klarenbeek :
                venue  frequency
0  Athletics

In [39]:
# Define function for returning the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
# create a new dataframe with the top 10 venues displayed for eacht neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mburg_grouped['Neighborhood']

for ind in np.arange(mburg_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mburg_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arnemuiden,Soccer Field,Gas Station,Train Station,Gym,Food Court,Basketball Court,Supermarket,Construction & Landscaping,Jewelry Store,Drugstore
1,Binnenstad,Café,Restaurant,Bar,Hotel,Plaza,Park,Concert Hall,Coffee Shop,Clothing Store,Seafood Restaurant
2,Dauwendaele,Hotel,Bus Stop,Fast Food Restaurant,Playground,Café,Intersection,Bowling Alley,Convenience Store,Farm,Furniture / Home Store
3,Griffioen,Fast Food Restaurant,Arcade,Chinese Restaurant,Supermarket,Electronics Store,Convenience Store,Cosmetics Shop,Department Store,Diner,Discount Store
4,Klarenbeek,Hotel,Athletics & Sports,Bakery,Wine Shop,Theater,Supermarket,Snack Place,Skate Park,Yoga Studio,Cosmetics Shop
5,Middelburg-Zuid,Supermarket,Shopping Mall,Furniture / Home Store,Cosmetics Shop,Flower Shop,Fast Food Restaurant,Hotel,Sporting Goods Shop,Bus Stop,Drugstore
6,Mortiere,Golf Course,Fast Food Restaurant,Toy / Game Store,Furniture / Home Store,Mini Golf,Restaurant,Bus Stop,Hotel Bar,Buffet,Paper / Office Supplies Store
7,Nieuw - en Sint Joosland,Golf Course,Soccer Field,Intersection,Sporting Goods Shop,French Restaurant,Electronics Store,Convenience Store,Cosmetics Shop,Department Store,Diner
8,Nieuw Middelburg,French Restaurant,Snack Place,Furniture / Home Store,Flower Shop,Fast Food Restaurant,Wine Shop,Gym / Fitness Center,Harbor / Marina,Drugstore,Hotel
9,Stromenwijk 't Zand,Chinese Restaurant,Athletics & Sports,Café,Basketball Court,Flower Shop,Supermarket,Sports Club,Discount Store,Snack Place,Drugstore


## Clustering

### clustering each neighborhood into five clusters using k-means clustering

In [41]:
# set number of clusters
kclusters = 5

mburg_grouped_clustering = mburg_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mburg_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 3, 0, 1, 2, 4, 1, 1], dtype=int32)

### generating new dataframe including the newly defined clusters for each neighborhood

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mburg_merged = df_data_mburg_neigh

# merge mburg_grouped with df_data_mburg_neigh
# to add latitude/longitude for each neighborhood
mburg_merged = mburg_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

mburg_merged

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Binnenstad,51.499732,3.613752,2,Café,Restaurant,Bar,Hotel,Plaza,Park,Concert Hall,Coffee Shop,Clothing Store,Seafood Restaurant
1,Griffioen,51.506202,3.595598,3,Fast Food Restaurant,Arcade,Chinese Restaurant,Supermarket,Electronics Store,Convenience Store,Cosmetics Shop,Department Store,Diner,Discount Store
2,Klarenbeek,51.508051,3.610783,0,Hotel,Athletics & Sports,Bakery,Wine Shop,Theater,Supermarket,Snack Place,Skate Park,Yoga Studio,Cosmetics Shop
3,Nieuw Middelburg,51.506233,3.625254,1,French Restaurant,Snack Place,Furniture / Home Store,Flower Shop,Fast Food Restaurant,Wine Shop,Gym / Fitness Center,Harbor / Marina,Drugstore,Hotel
4,Veersepoort,51.512197,3.622476,1,Athletics & Sports,Supermarket,Plaza,Soccer Field,Flower Shop,Fast Food Restaurant,Drugstore,Pharmacy,Skate Park,Furniture / Home Store
5,Stromenwijk 't Zand,51.491164,3.599305,1,Chinese Restaurant,Athletics & Sports,Café,Basketball Court,Flower Shop,Supermarket,Sports Club,Discount Store,Snack Place,Drugstore
6,Middelburg-Zuid,51.484007,3.614082,1,Supermarket,Shopping Mall,Furniture / Home Store,Cosmetics Shop,Flower Shop,Fast Food Restaurant,Hotel,Sporting Goods Shop,Bus Stop,Drugstore
7,Dauwendaele,51.492077,3.625996,2,Hotel,Bus Stop,Fast Food Restaurant,Playground,Café,Intersection,Bowling Alley,Convenience Store,Farm,Furniture / Home Store
8,Mortiere,51.483405,3.635964,2,Golf Course,Fast Food Restaurant,Toy / Game Store,Furniture / Home Store,Mini Golf,Restaurant,Bus Stop,Hotel Bar,Buffet,Paper / Office Supplies Store
9,Nieuw - en Sint Joosland,51.481970,3.656884,4,Golf Course,Soccer Field,Intersection,Sporting Goods Shop,French Restaurant,Electronics Store,Convenience Store,Cosmetics Shop,Department Store,Diner


## Visualize new results

In [43]:
# create map
map_clusters = folium.Map(location=lat_long_mburg_nl, zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mburg_merged['Latitude'], 
                                  mburg_merged['Longitude'], 
                                  mburg_merged['Neighborhood'], 
                                  mburg_merged['Cluster Labels']):
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## View Cluster one (1)

In [44]:
# Output cluster one
mburg_merged.loc[mburg_merged['Cluster Labels'] == 0, 
                 mburg_merged.columns[[0] + list(range(4, mburg_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Klarenbeek,Hotel,Athletics & Sports,Bakery,Wine Shop,Theater,Supermarket,Snack Place,Skate Park,Yoga Studio,Cosmetics Shop


## View Cluster two (2)

In [45]:
# Output cluster two
mburg_merged.loc[mburg_merged['Cluster Labels'] == 1, 
                 mburg_merged.columns[[0] + list(range(4, mburg_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Nieuw Middelburg,French Restaurant,Snack Place,Furniture / Home Store,Flower Shop,Fast Food Restaurant,Wine Shop,Gym / Fitness Center,Harbor / Marina,Drugstore,Hotel
4,Veersepoort,Athletics & Sports,Supermarket,Plaza,Soccer Field,Flower Shop,Fast Food Restaurant,Drugstore,Pharmacy,Skate Park,Furniture / Home Store
5,Stromenwijk 't Zand,Chinese Restaurant,Athletics & Sports,Café,Basketball Court,Flower Shop,Supermarket,Sports Club,Discount Store,Snack Place,Drugstore
6,Middelburg-Zuid,Supermarket,Shopping Mall,Furniture / Home Store,Cosmetics Shop,Flower Shop,Fast Food Restaurant,Hotel,Sporting Goods Shop,Bus Stop,Drugstore
10,Arnemuiden,Soccer Field,Gas Station,Train Station,Gym,Food Court,Basketball Court,Supermarket,Construction & Landscaping,Jewelry Store,Drugstore


## View Cluster three (3)

In [46]:
# Output cluster three
mburg_merged.loc[mburg_merged['Cluster Labels'] == 2, 
                 mburg_merged.columns[[0] + list(range(4, mburg_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Binnenstad,Café,Restaurant,Bar,Hotel,Plaza,Park,Concert Hall,Coffee Shop,Clothing Store,Seafood Restaurant
7,Dauwendaele,Hotel,Bus Stop,Fast Food Restaurant,Playground,Café,Intersection,Bowling Alley,Convenience Store,Farm,Furniture / Home Store
8,Mortiere,Golf Course,Fast Food Restaurant,Toy / Game Store,Furniture / Home Store,Mini Golf,Restaurant,Bus Stop,Hotel Bar,Buffet,Paper / Office Supplies Store


## View Cluster four (4)

In [47]:
# Output cluster four
mburg_merged.loc[mburg_merged['Cluster Labels'] == 3, 
                 mburg_merged.columns[[0] + list(range(4, mburg_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Griffioen,Fast Food Restaurant,Arcade,Chinese Restaurant,Supermarket,Electronics Store,Convenience Store,Cosmetics Shop,Department Store,Diner,Discount Store


## View Cluster five (5)

In [48]:
# Output cluster five
mburg_merged.loc[mburg_merged['Cluster Labels'] == 4, 
                 mburg_merged.columns[[0] + list(range(4, mburg_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Nieuw - en Sint Joosland,Golf Course,Soccer Field,Intersection,Sporting Goods Shop,French Restaurant,Electronics Store,Convenience Store,Cosmetics Shop,Department Store,Diner


# Final Conclusion

In the beginning the following question was defined: 
- In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it?

The city of my choice was obviously Middelburg, now the question depends on what type of restaurant there needs to be placed, such as a Chinese restaurant for example and if there's need for a particular restaurant in the neighborhood.

After examening the five clusters I have came to the following conclusing*:

Cluster:
1. Cluster one has a snack place, an Asian restaurant, and a Fast food restaurant, looking at the most common venues I would consider that a different type than an asian Restaurant or fastfood restaurant will be welcome in this neighborhood, however there doens't seem to be a lot of restaurant, so it isn't a populair place. I would consider cluster one a residential neighborhood.


2. Cluster two has a lot more restaurants and café's than cluster one for example, I would consider this the outskirts neighborhoods of Middelburg because it got everything you need, depending on the type of restaurant there will be need. There's less need for a snack place, fast-food restaurant, and Chinese/French restaurant's.


3. Cluster three seems to be the best place to start a new restaurant because of the large variety of restaurant's and it's localisation as a city centre type of neighborhoods, also more expensive restaurant's would be more suitable here because of the luxury activities such as golf for example 


4. Cluster four would also be a great place to start a new restaurant targeting younger people, there seem to be mostly younger people as there are dancing studio's and arcade game halls as well


5. Cluster five  would be considered a place for the more quieter restaurants as it is (far) more outside of the centre of Middelburg.

Cluster three seems to have the best neighborhoods in Middelburg for a (new) Restaurant.

*For a complete and more detailed conlusion see the full report.